In [ ]:
# from dashboard import Dashboard
# from sidecar import Sidecar
import numpy as np
from skimage.io import imread, imsave
from scipy import sparse
# from matplotlib import pyplot as plt
import plotly.graph_objs as go

from IPython.display import display
from ipywidgets import Accordion, FileUpload, Label, Output, IntProgress, IntSlider, Tab, IntText, Text, Button, Dropdown, AppLayout, Box, SelectionSlider, FloatSlider, Button, HBox, VBox
from traitlets import default, List, HasTraits
from analysis_driver import AnalysisDriver
from visible_optimizer import VisibleOptimizer



# def rebinnable_interactive_histogram(series, bins, initial_bin_width=10):
#     start = bins[0]
#     end = bins[-1]
#     size = (end-start)/(len(bins)-1)

#     figure_widget = go.FigureWidget(
#         data=[go.Histogram(x=series, xbins={"start": start, "size": size, "end": end})] # {"size": initial_bin_width})]
#     )
#     bin_slider = FloatSlider(
#         value=initial_bin_width,
#         min=1,
#         max=30,
#         step=1,
#         description="Bin width:",
#         readout_format=".0f",  # display as integer
#     )
#     histogram_object = figure_widget.data[0]
#     def set_bin_size(change):
#         pass
#         # histogram_object.xbins = {"size": change["new"]}
#     bin_slider.observe(set_bin_size, names="value")
#     output_widget = VBox([figure_widget, bin_slider])
#     return output_widget

        
        
class Dashboard(HasTraits):
    project_list = List()
    
    def __init__(self):
        self.progress_bars: List[IntProgressBar] = {}
        self._file_analysis: Dict = {}
                
        rnd_data = np.random.normal(loc=50.0, scale=18.5, size=[2800])
        histo, bin_edges = np.histogram(rnd_data, bins=18, density=False)
#         histo_widget = rebinnable_interactive_histogram(histo, bin_edges)
#         print(histo_widget)

        self.t_file_upload = FileUpload(description='Upload an image for Analysis')
        self.width_field = IntText(description='Image Width', value=2626)
        self.height_field = IntText(description='Image Height', value=3674)
        self.header = HBox(children=[self.t_file_upload, self.width_field, self.height_field], height="100%")
        self.debug_output = Output()
        self.progress_box = HBox(children=[])
        self.tab_analysis_results = Tab(children=[
            Text("TBD")
        ])
        self.acc_open_files = Accordion(
            description='View Options', 
            children=[self.progress_box, self.tab_analysis_results, self.debug_output])
                      # histo_widget, self.debug_output])
        
        self.center_panel = VBox(children=[self.acc_open_files])
        self.right_panel = VBox(children=[])
        self.app_layout = AppLayout(
             header=self.header, left_sidebar=None, center=self.center_panel, right_sidebar=self.right_panel, 
             pane_widths=["3%", "72%", "25%"], pane_heights=["15%", "80%", "5%"], footer=None, height="480px")
#         self.app_tab = Stacked(children=[self.app_layout, self.debug_output])
#         self.app_tab.titles=["Main", "Debug"]
        
        self.app_layout = VBox(children=[self.header, self.acc_open_files], height='480px')
        
    @default('project_list')
    def default_project_list(self):
        sseturn [('John', 0), ('Jamie', 1), ('Kay', 2)]
        
    def display(self):
        display(self.app_layout)
        display(self.debug_output)
        
    def on_file_uploaded(self, e):
        width = self.width_field.value
        height = self.height_field.value
        upload_record = self.t_file_upload.value
        for key in upload_record:
            if not key in self._file_analysis:
                progress_bar = IntProgress(
                    value=0, min=0, max=8, step=1, 
                    description=f"Optimizing: {key}", 
                    orientation='horizontal',
                    bar_style='info')
                upload_content = upload_record[key]["content"]
                analyzer = VisibleOptimizer(key, upload_content, progress_bar, width=width, height=height)
                self._file_analysis[key] = {
                    "progress_bar": progress_bar,
                    "file_content": upload_content,
                    "visual_optimizer": analyzer
                }
                with self.debug_output as dbg:
                    print(f"Analysis in progress for {key}")
                #self.center_panel.children.append(0, progress_bar)
                display(progress_bar)
                analyzer.run_analysis()
                results = analyzer.collect_widgets()
                self._file_analysis[key]["results"] = results
                del self._file_analysis[key]["file_content"]
                progress_bar.value = 10
                self.center_panel.children.reverse()
                self.center_panel.children.pop()
                self.center_panel.children.reverse()
                self.acc_open_files.children.append(results["widget"])

        
main = Dashboard()
main.display()
main.t_file_upload.observe(main.on_file_uploaded, 'value')

hits = 0

def handler2(e):
    global hits
    hits = hits + 1
    
main.t_file_upload.observe(handler2, 'value')


In [ ]:
key = "900_accum16_20200212_163257.png"
output = Output(height="200px")
progress_bar = IntProgress(
        value=-1, min=0, max=10, step=1, 
        description=f"Optimizing: {key}", 
        orientation='horizontal',
        bar_style='info')
progress_bar.value = 50
display(output)
display(progress_bar)
print("Bar")
with output as op:
    print("Foo")
print("Bam")

In [ ]:
from skimage.io import imread
import numpy as np

with output as op:
    path = "./ee19f416b1735c6ec5fc2ff3c1524a761032d001/900_accum16_20200212_163257.png"
    key = "900_accum16_20200212_163257.png"
    data = imread(path)
    d = data.flatten()
    print(d.shape)
    
    f = AnalysisDriver(key, d, progress_bar, width=1512, height=2016) 
    display(progress_bar)
    f.run_analysis()
    



In [ ]:
import ipywidgets

print(dir(ipywidgets))